<a href="https://colab.research.google.com/github/Qurat-ul-Ain-Gul19/PIAIC-QUARTER-3/blob/main/COL-Copy_of_Chapter8_8_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
def reweight_distribution(original_distribution, temperature=0.5):
  distribution = np.log(original_distribution) / temperature
  distribution = np.exp(distribution)
  return distribution / np.sum(distribution)

In [ ]:
text = """
After being tested COVID positive on 2 April 2021, I was rushed to Pakistan
Emirates Military Hospital (PEMH) Rawalpindi where I was admitted urgently,
then followed by my wife, after two days. I would like to share the painful
experience of symptoms related to the disease and how the wonderful and
caring hospital staff took care of us. It is for the sake of posterity and letting
the readers know about the good work being done by medical establishments
run by Pakistan Armed Forces and about which very few people actually know
about.
My experience as a patient has
touched me so deeply that I have
been inspired to write this TRIBUTE,
realising that not only are they the
most unique but truly the HEROES
against the monstrous enemy of the
humanity.Covid-19 has brought a murderous and menacing holocaust of the size never
been experienced in the history of mankind, except the Spanish Flu, affecting
every nation across the globe. For once, one saw the human race coming
forward to show camaraderie and unity to face the monster that has invaded
every home in every nook and corner of the world. The mask, protective
gloves and rest of the Corona protective gear has become a part of everyday
apparel and standard SOP as part of our living. International air travel has
reduced global village to abodes of isolation.
Where the scientists have been trying to get to know the origins of the virus
and find a breakthrough in treatment, it has been a challenge for the brave
doctors, paramedics, nursing and logistics staff, who faced the pandemic, living
and working in an environment that was overflowing with the “virus of death”.
They never failed to live up to the expectations offering their lives in the
process as the hospitals started overflowing with the suffering humanity,
irrespective of sex and age. This type of human behavior had only been
witnessed in natural calamities
From the moment I got into the hospital, the practical application of terms
“sacrifice” and “beyond the call of duty”, was visible all around. This is the kind
of stuff I had experienced in raging tank battles during my early army life.
As soon as I was into the hospital, I could feel a sense of urgency in everyone’s
demeanour and despite heavy workload, lack of staff and minor inadequacies,
the staff ably led by Brig Raja Khalid Mehmood, the Nephrologist and his highly
efficient team of experienced doctors, nursing staff, paramedics and the
logistics staff put in their best to attend to every patient in the best
professional manner. The memory of the angelic faces of the entire hospital
staff will always stay fresh in my mind, reminding me of the loving care, as one
can only expect from one’s family. No amount of gratitude will suffice in
acknowledging their deeds.
While I was in the hospital, I kept learning of fatalities taking place on daily
basis, which in itself is so depressing for any patient. Amongst them was one
of the noblest and revered souls, Lt Col Mahjabeen Riaz, Wife of Lt Gen Riaz
Chohan, a former Surgeon General of Pakistan Army. May Allah rest her soul in
peace and in heaven. Like me and my wife, they were also hospitalized
together. I have known Lt Gen Riaz Chohan, since my youth.


""".lower()
print(len(text))
text

3215


'\nafter being tested covid positive on 2 april 2021, i was rushed to pakistan\nemirates military hospital (pemh) rawalpindi where i was admitted urgently,\nthen followed by my wife, after two days. i would like to share the painful\nexperience of symptoms related to the disease and how the wonderful and\ncaring hospital staff took care of us. it is for the sake of posterity and letting\nthe readers know about the good work being done by medical establishments\nrun by pakistan armed forces and about which very few people actually know\nabout.\nmy experience as a patient has\ntouched me so deeply that i have\nbeen inspired to write this tribute,\nrealising that not only are they the\nmost unique but truly the heroes\nagainst the monstrous enemy of the\nhumanity.covid-19 has brought a murderous and menacing holocaust of the size never\nbeen experienced in the history of mankind, except the spanish flu, affecting\nevery nation across the globe. for once, one saw the human race coming\nfor

## Listing 8.2 Downloading and parsing the initial text file

In [ ]:
import keras
import numpy as np
path = keras.utils.get_file('nietzsche.txt',
                            origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')

text = open(path).read().lower()
print('Corpus length:', len(text))

606208/600901 [==============================] - 0s 0us/step
Corpus length: 600893


## Listing 8.3 Vectorizing sequences of characters

In [ ]:
maxlen = 60
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
  sentences.append(text[i: i + maxlen])
  next_chars.append(text[i + maxlen])

In [ ]:

print('Number of sequences:', len(sentences))
chars = sorted(list(set(text)))
print('Unique characters:', len(chars))
char_indices = dict((char, chars.index(char)) for char in chars)
print('Vectorization...')


Number of sequences: 200278
Unique characters: 57
Vectorization...


In [ ]:

x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)

y[2]

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False])

In [ ]:

for i, sentence in enumerate(sentences):
  for t, char in enumerate(sentence):
    x[i, t, char_indices[char]] = 1
  y[i, char_indices[next_chars[i]]] = 1

In [ ]:
y[0]

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False,  True,
       False, False, False, False, False, False, False, False, False,
       False, False, False])

## Listing 8.4 Single-layer LSTM model for next-character prediction

In [ ]:
import keras
from keras import layers
model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))

In [ ]:
optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

## Listing 8.6 Function to sample the next character given the model’s predictions

In [ ]:
def sample(preds, temperature=1.0):
  preds = np.asarray(preds).astype('float64')
  preds = np.log(preds) / temperature
  exp_preds = np.exp(preds)
  preds = exp_preds / np.sum(exp_preds)
  probas = np.random.multinomial(1, preds, 1)
  return np.argmax(probas)

## Listing 8.7 Text-generation loop

In [ ]:
import random
import sys


model.fit(x, y, batch_size=128, epochs=10)
start_index = random.randint(0, len(text) - maxlen - 1)
generated_text = text[start_index: start_index + maxlen]
print('--- Generating by Machine: "' + generated_text + '"')

Epoch 1/10
1565/1565 [==============================] - 42s 6ms/step - loss: 2.2643
Epoch 2/10
1565/1565 [==============================] - 10s 6ms/step - loss: 1.6310
Epoch 3/10
1565/1565 [==============================] - 10s 6ms/step - loss: 1.5248
Epoch 4/10
1565/1565 [==============================] - 10s 6ms/step - loss: 1.4761
Epoch 5/10
1565/1565 [==============================] - 10s 6ms/step - loss: 1.4380
Epoch 6/10
1565/1565 [==============================] - 10s 6ms/step - loss: 1.4117
Epoch 7/10
1565/1565 [==============================] - 10s 6ms/step - loss: 1.3971
Epoch 8/10
1565/1565 [==============================] - 10s 6ms/step - loss: 1.3811
Epoch 9/10
1565/1565 [==============================] - 10s 6ms/step - loss: 1.3738
Epoch 10/10
1565/1565 [==============================] - 10s 6ms/step - loss: 1.3623
--- Generating by Machine: "like the sun. and we foremost, we good europeans!

244. ther"


In [ ]:
temperature = 0.5
for i in range(400):
    sampled = np.zeros((1, maxlen, len(chars)))
    for t, char in enumerate(generated_text):
      sampled[0, t, char_indices[char]] = 1.
    preds = model.predict(sampled, verbose=0)[0]
    next_index = sample(preds, temperature)
    next_char = chars[next_index]
    generated_text += next_char
    generated_text = generated_text[1:]
    sys.stdout.write(next_char)

e is there the present that it is say, the defection and discovered love of a dependent has seems to existence, to makes to be greated new to any one must be that it cannot been a present and self-destruction of the content of the for a man of the spirit and the belief in the free spirit to disself for the exception and stands a very one of the world is as the subjection the precipilition that sou

In [ ]:
import random
import sys

for epoch in range(1, 60):
  print('epoch', epoch)
  model.fit(x, y, batch_size=128, epochs=1)
  start_index = random.randint(0, len(text) - maxlen - 1)
  generated_text = text[start_index: start_index + maxlen]
  print('--- Generating with seed: "' + generated_text + '"')

for temperature in [0.2, 0.5, 1.0, 1.2]:
  print('------ temperature:', temperature)
  sys.stdout.write(generated_text)
  for i in range(400):
    sampled = np.zeros((1, maxlen, len(chars)))
    for t, char in enumerate(generated_text):
      sampled[0, t, char_indices[char]] = 1.
    preds = model.predict(sampled, verbose=0)[0]
    next_index = sample(preds, temperature)
    next_char = chars[next_index]
    generated_text += next_char
    generated_text = generated_text[1:]
    sys.stdout.write(next_char)

epoch 1
1565/1565 [==============================] - 10s 6ms/step - loss: 1.3690
--- Generating with seed: "ssimilation of its nutriment. there are honestly meant trans"
epoch 2
1565/1565 [==============================] - 10s 6ms/step - loss: 1.3601
--- Generating with seed: " and the sign of a despairing,
mortally wearied soul, notwit"
epoch 3
1565/1565 [==============================] - 10s 6ms/step - loss: 1.3533
--- Generating with seed: "rent series of feeling and thoughts, whereas it is
unconscio"
epoch 4
1565/1565 [==============================] - 10s 6ms/step - loss: 1.3475
--- Generating with seed: "l states being falsely
viewed and his personality being sund"
epoch 5
1565/1565 [==============================] - 10s 6ms/step - loss: 1.3425
--- Generating with seed: "rder of
philosophers, such as will have other tastes and inc"
epoch 6
1565/1565 [==============================] - 10s 6ms/step - loss: 1.3371
--- Generating with seed: "ropound an
explanation, too, free from myt